# SPD

Replicate the design of [Smaldino et al. (2013)](https://www.journals.uchicago.edu/doi/10.1086/669615).

Add per-agent memory. Remember defectors. Vary memory size.

Add local communication: shout to nearby neighbors. Vary range & size.

## Imports

In [ ]:
# model
from mesa_fork import Model, Agent
from mesa_fork.time import RandomActivation
from mesa_fork.space import SingleGrid
from mesa_fork.datacollection import DataCollector
from enum import Enum

# visualization
import numpy as np
import matplotlib.pyplot as plt
plt.rc('axes', labelsize=8)
%matplotlib inline
plt.style.use('seaborn')
import holoviews as hv
%load_ext holoviews.ipython
import seaborn as sns
sns.set_theme(style="ticks")

from my_plot import my_plot_export

# parameter sweep
from mesa_fork.batchrunner import BatchRunnerMP

## Setup model

In [ ]:
class Memory:
    """
    Fixed-size FIFO (first-in-first-out) memory.
    Used by agents to remeber the most recent defectors.
    """
    
    def __init__(self,
                 size):
        """
        Args:
            size: memory size
        """
        
        self.size = size
        self.memory = []
        
        
    def add(self, item):
        
        # remove duplicates
        self.memory[:] = list(filter(
            lambda x: x != item,
            self.memory))
        
        self.memory.insert(0, item)
        
        # truncate to size
        while len(self.memory) > self.size:
            self.memory.pop(-1)
       
    
    def contains(self, item):
        try:
            self.memory.index(item)
            return True
        except ValueError:
            return False
        
        
    def get(self):
        
        return self.memory.copy()
            

In [ ]:
class Action(Enum):
    COOPERATE = 1
    DEFECT    = 2

    
class MyAgent(Agent):
    
    def __init__(self, 
                 model,
                 energy,
                 max_energy,
                 memory_size,
                 gossip_range,
                 gossip_size,
                 cooperator=False):
        """
        Agent behaviour
        
        Args:
            model:         reference to the model containing this agent
            energy:        starting energy level
            max_energy:    maximal energy limit
            memory_size:   size of agent memory
            gossip_range:  maximal range at which this agent can gossip
            gossip_size:   maximal amount of information which can be gossip in a single round
        """
        
        super().__init__(model.next_id(), model)
        
        self.memory = Memory(memory_size)
        
        self.energy = energy
        self.max_energy = max_energy
        self.cooperator = cooperator
        
        self.gossip_range = gossip_range
        self.gossip_size = gossip_size
        
        self.played = True
        self.newborn = True
        
        
    def play(self):
        """
        Return game action based on agent type.
        All agents use either always-cooperate or always-defect strategy.
        """
        
        if self.cooperator:
            return Action.COOPERATE
        else:
            return Action.DEFECT
        
    
    def gossip(self):
        """
        Return gossip about remebered defectors
        """
        
        gossip = self.memory.get()
        
        # truncate to size
        while len(gossip) > self.gossip_size:
            gossip.pop(-1)
        
        return gossip
    
    
    def will_play_with(self, opponent):
        """
        Return True if willing to play with the opponent
        Otherwise False
        """
        
        # can only play once a turn
        if self.played:
            return False
        
        # check own memory
        if self.memory.contains(opponent):
            return False
        
        # get gossip
        gossip_knowledge = set()
        gossipers = self.model.grid.get_neighbors(self.pos, moore=True, radius=self.gossip_range)
        for gossiper in gossipers:
            gossip_knowledge.update(gossiper.gossip())
        # check gossip about opponent
        if opponent in gossip_knowledge:
            return False
        
        # opponent does not have bad reputation
        return True
        
        
    def step(self):
        """
        Agent behaviour in a single timestep.
        """
        
        # don't step if created this turn
        if self.newborn:
            return
        
        # get eligible opponents
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True)
        # discard neighbors who have already played a game in this step
        opponents = list(filter(
            lambda a: self.will_play_with(a.unique_id) and a.will_play_with(self.unique_id),
            neighbors))

        # find opponent
        if opponents:
            opponent = self.random.choice(opponents)

            # play pd game
            if not self.played:

                a = self.play()
                b = opponent.play()

                R, T, S, P = self.model.R, self.model.T, self.model.S, self.model.P

                if   a == Action.COOPERATE and b == Action.COOPERATE:
                    self.energy     += R
                    opponent.energy += R
                    
                elif a == Action.COOPERATE and b == Action.DEFECT:
                    self.energy     += S
                    opponent.energy += T

                    # remember betrayal
                    self.memory.add(opponent.unique_id)
                    
                elif a == Action.DEFECT    and b == Action.COOPERATE:
                    self.energy     += T
                    opponent.energy += S

                    # remember betrayal
                    opponent.memory.add(self.unique_id)
                    
                elif a == Action.DEFECT    and b == Action.DEFECT:
                    self.energy     += P
                    opponent.energy += P

                    # remember betrayals
                    self.memory.add(opponent.unique_id)
                    opponent.memory.add(self.unique_id)
                    
                self.energy = min(self.energy, self.max_energy)
                opponent.energy = min(opponent.energy, opponent.max_energy)

                self.played = True
                opponent.played = True

            # reproduce
            max_population = (self.model.grid.width * self.model.grid.height) / 2
            if (    self.cooperator and self.model.last_cooperator_count < max_population) or \
               (not self.cooperator and self.model.last_defector_count   < max_population):

                neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
                unoccupied = list(filter(
                    lambda c: self.model.grid.is_cell_empty(c), 
                    neighborhood))

                if self.energy >= (self.model.energy_to_reproduce * 2) and unoccupied:

                    cell = self.random.choice(unoccupied)
                    offspring = MyAgent(self.model,
                                        energy=self.model.energy_to_reproduce,
                                        max_energy=self.max_energy,
                                        cooperator=self.cooperator,
                                        memory_size=self.memory.size,
                                        gossip_size=self.gossip_size,
                                        gossip_range=self.gossip_range)
                    self.model.grid.position_agent(offspring, cell[0], cell[1])
                    self.model.schedule.add(offspring)

                    # update values for DataCollector
                    self.model.agent_count += 1
                    if self.cooperator:
                        self.model.cooperator_count += 1
                    else:
                        self.model.defector_count += 1

                    self.energy -= self.model.energy_to_reproduce


        elif not self.played:
            # attempt movement
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
            unoccupied = list(filter(
                lambda c: self.model.grid.is_cell_empty(c), 
                neighborhood))

            if unoccupied:
                cell = self.random.choice(unoccupied)
                self.model.grid.move_agent(self, cell)


        # energy deduction (cost of living)
        self.energy -= self.model.living_cost
        if self.energy <= 0:
            # die
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
            
        # update values for DataCollector
        self.model.agent_count += 1

        if self.cooperator:
            self.model.cooperator_count += 1
        else:
            self.model.defector_count += 1


In [ ]:
class SPDModel(Model):
    
    def __init__(self,
                 R=3, T=5, S=-1, P=0,
                 starting_agent_count=10,
                 starting_energies=range(1,51),
                 max_energy=150,
                 energy_to_reproduce=50,
                 living_cost=1,
                 memory_size=0,
                 gossip_range=0,
                 gossip_size=0,
                 grid_size=10,
                 wrap=True):
        """
        Smaldino's spatial prisonner's dilemma model
        extended with limited memory
        
        Args:
            R, T, S, P:            PD payoffs
            starting_agent_count:  starting number of agents
            starting_energies:     list of possible starting energies for agents (picked at random)
            max_energy:            maximal energy an agent can hold
            energy_to_reproduce:   energy required to reproduce
            living_cost:           energy deducted at the end of each step
            memory_size:           agent memory size
            gossip_range:          neighborhood range for gossip
            gossip_size:           amount of info to gossip
            grid_size:             size length of square grid to use
            wrap:                  whether to wrap grid (torus bounds)
        """
        
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(grid_size, grid_size, torus=wrap)
        
        self.R = R
        self.T = T
        self.S = S
        self.P = P
        self.energy_to_reproduce = energy_to_reproduce
        self.living_cost = living_cost
        
        # Setup agents
        self.cooperator_count = 0
        self.defector_count = 0

        for i in range(starting_agent_count):
            energy = self.random.choice(starting_energies)
            cooperator = i%2 == 0
            
            if cooperator:
                self.cooperator_count += 1
            else:
                self.defector_count += 1
            
            agent = MyAgent(self, 
                            energy,
                            max_energy=max_energy,
                            cooperator=cooperator,
                            memory_size=memory_size,
                            gossip_range=gossip_range,
                            gossip_size=gossip_size)
        
            cell = self.random.choice(list(self.grid.empties))

            self.grid.position_agent(agent, cell[0], cell[1])
            self.schedule.add(agent)
        
        self.agent_count = starting_agent_count
        
        # Init model
        self.running = True
        
        self.datacollector = DataCollector(
            {
                "agent_count": "agent_count",
                "cooperator_count": "cooperator_count",
                "defector_count": "defector_count",
            },
        )
        self.datacollector.collect(self)
        
        
    def step(self):
        
        # setup for step
        self.last_cooperator_count = self.cooperator_count
        self.last_defector_count   = self.defector_count
        
        self.agent_count = 0
        self.cooperator_count = 0
        self.defector_count = 0

        for a in self.schedule.agents:
            a.played = False
            a.newborn = False
    
        # step
        self.schedule.step()
        self.datacollector.collect(self)
        
        # stop the model if no agents are alive
        if self.agent_count == 0:
            self.running = False


## Run model

In [ ]:
spd = SPDModel(R=3, T=5, S=-1.5, P=0,
               starting_agent_count=64,
               starting_energies=range(1,50),
               max_energy=150,
               energy_to_reproduce=50,
               living_cost=0.5,
               memory_size=1,
               gossip_size=0,
               gossip_range=0,
               grid_size=20,
               wrap=True)

In [ ]:
i = 0
while spd.running and i < 500:
    spd.step()
    i += 1

### Check results

In [ ]:
max_population = (spd.grid.width * spd.grid.height) / 2

results = spd.datacollector.get_model_vars_dataframe()
results['cooperator_saturation'] = (results['cooperator_count'] / max_population)
results['defector_saturation']   = (results['defector_count']   / max_population)

fig, ax = plt.subplots(1, 1)
sns.lineplot(data=results[['cooperator_saturation', 'defector_saturation']], ax=ax)
# ax.set_xlim(0, 1000)
ax.set_ylim(-0.1, 1.1)
ax.set_xlabel('step')
ax.set_ylabel('agent type saturation')

In [ ]:
my_plot_export(fig, [ax], 'saturation&step-memory1+gossip1+range3')

### Render visualization

In [ ]:
def value(cell):
    if cell is None:
        return 0
    elif isinstance(cell, Agent):
        if cell.cooperator:
            return 2
        else:
            return 10
    else:
        raise Exception("Unidentified cell: {}".format(cell))

hmap = hv.HoloMap(kdims='step')
i = 0
while spd.running and i < 20:
    spd.step()
    data = np.array([[value(c) for c in row] for row in spd.grid.grid])
    hmap[i] = hv.Image(data, vdims=[hv.Dimension('State', range=(0,10))])
    i += 1
hmap

## Paramater sweep

In [ ]:
variable_parameters = {
    "living_cost":           np.linspace(0.0, 3.5, num=10),
}
fixed_parameters = {
    "gossip_size": 1,
    "memory_size": 1,
    "gossip_range": 3,
    "R": 3,
    "T": 5,
    "S": -1.5,
    "P": 0,
    "starting_agent_count":  64,
    "starting_energies":     range(1,50),
    "max_energy":            150,
    "energy_to_reproduce":   50,
    "grid_size":             20,
    "wrap":                  True,
}

iterations = 30
max_steps = 1000

param_run = BatchRunnerMP(SPDModel,
                          nr_processes=None,  # detect automatically
                          variable_parameters=variable_parameters,
                          fixed_parameters=fixed_parameters,
                          iterations=iterations,
                          max_steps=max_steps,
                          model_reporters={
                              "agent_count": lambda m: m.agent_count,
                              "cooperator_count": lambda m: m.cooperator_count,
                              "defector_count": lambda m: m.defector_count,
                          })

param_run.run_all()

max_population = (fixed_parameters["grid_size"] ** 2) / 2

run_data = param_run.get_model_vars_dataframe()
run_data['cooperator_saturation'] = (run_data['cooperator_count'] / max_population)
run_data['defector_saturation']   = (run_data['defector_count']   / max_population)
run_data = run_data.dropna()
# run_data.head()

In [ ]:
PROPS = {
    'boxprops':{'facecolor':'lightgrey', 'edgecolor':'black'},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

fig, axs = plt.subplots(1, 2, figsize=(16, 3))
plt.suptitle("Agent type saturation after {} steps".format(max_steps), fontsize=16)

sns.boxplot(x="living_cost", y="cooperator_saturation",
            data=run_data, ax=axs[0],
            showfliers=False, **PROPS)
sns.boxplot(x="living_cost", y="defector_saturation",
            data=run_data, ax=axs[1],
            showfliers=False, **PROPS)

for ax in axs:
    ax.set_xlabel("Cost of living")
    ax.set_ylim(-0.1, 1.1)
    labels = [item.get_text() for item in ax.get_xticklabels()]
    ax.set_xticklabels([str(round(float(label), 2)) for label in labels])

axs[0].set_ylabel("Cooperator saturation")
axs[1].set_ylabel("Defector saturation")

In [ ]:
name = 'saturation&living_cost-memory{}_gossip{}_range{}_{}steps'.format(
        fixed_parameters["memory_size"], 
        fixed_parameters["gossip_size"], 
        fixed_parameters["gossip_range"], 
        max_steps)

my_plot_export(fig, axs, name)
my_plot_export(fig, axs, '{}_large'.format(name), fontsize=16, width=16, height=3)